In [7]:
import json
import logging
import operator
import pprint
import time

import boto3
from botocore.exceptions import ClientError

logger = logging.getLogger(__name__)
iam = boto3.resource('iam')

In [10]:
def create_role(role_name, allowed_services):
    """
    Creates a role that lets a list of specified services assume the role.
    :param role_name: The name of the role.
    :param allowed_services: The services that can assume the role.
    :return: The newly created role.
    """
    trust_policy = {
        'Version': '2012-10-17',
        'Statement': [{
                'Effect': 'Allow',
                'Principal': {'Service': service},
                'Action': 'sts:AssumeRole'
            } for service in allowed_services
        ]
    }

    try:
        role = iam.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(trust_policy))
        logger.info("Created role %s.", role.name)
    except ClientError:
        logger.exception("Couldn't create role %s.", role_name)
        raise
    else:
        return role

In [12]:
def attach_policy(role_name, policy_arn):
    """
    Attaches a policy to a role.
    :param role_name: The name of the role. **Note** this is the name, not the ARN.
    :param policy_arn: The ARN of the policy.
    """
    try:
        iam.Role(role_name).attach_policy(PolicyArn=policy_arn)
        logger.info("Attached policy %s to role %s.", policy_arn, role_name)
    except ClientError:
        logger.exception("Couldn't attach policy %s to role %s.", policy_arn, role_name)
        raise

In [15]:
role_name = "lambda-vpc-role"

In [16]:
create_role(role_name, ['lambda.amazonaws.com'])

iam.Role(name='lambda-vpc-role')

In [17]:
attach_policy(role_name, 'arn:aws:iam::aws:policy/service-role/AWSLambdaVPCAccessExecutionRole')